In [1]:
import requests
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import folium

In [11]:
def get_api_data(api_url:str):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    
    return None


def get_city_geolocation(city_name:str):
  api_key = "d1af9eb638a7a24e06f0889f9e2c6a0d"
  api_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city_name}&appid={api_key}"

  data = get_api_data(api_url)

  return {"latitude": data[0]['lat'], "longitude": data[0]['lon']}


def load_file_data(dataset_path:str, dataset_name:str):
    csv_path = os.path.join(dataset_path, dataset_name)
    return pd.read_csv(csv_path)

def filter_data(database):

    city = []
    confirmed = []
    deaths = []
    population = []
    death_rate = []
    latitude = []
    longitude = []
    
    for data in database.values:
        if data[0] == "2022-03-26" and data[1] == "PI":
            geolocation = get_city_geolocation(data[2])

            #print(f"{data[2]}({data[1]}) - confirmed:{data[4]}, deaths:{data[5]}, population:{data[7]}, death_hate: {data[10]}, lat: {geolocation['latitude']}, lon: {geolocation['longitude']}")

            city.append(data[2])
            confirmed.append(data[4])
            deaths.append(data[5])
            population.append(data[7])
            death_rate.append(data[10])
            latitude.append(geolocation['latitude'])
            longitude.append(geolocation['longitude'])

    return pd.DataFrame({
        "city": city,
        "confirmed": confirmed,
        "deaths": deaths,
        "population": population, 
        "death_hate": death_rate, 
        "latitude": latitude, 
        "longitude": longitude
    })

In [13]:
covid_database = load_file_data("/home/jcsf/Dev/Sistemas Inteligentes/Atividade Visualização de casos de COVID-19 no Piauí", "covid19database.csv")

covid_filtered_data = filter_data(covid_database)

covid_filtered_data.head(5)

,city,confirmed,deaths,population,death_hate,latitude,longitude
0,Acauã,228,2,7102.0,0.0088,-8.219542,-41.083059
1,Agricolândia,829,10,5131.0,0.0121,-5.796784,-42.660513
2,Água Branca,1702,77,17470.0,0.0452,-5.891392,-42.636115
3,Alagoinha do Piauí,402,13,7665.0,0.0323,-7.009095,-40.939675
4,Alegrete do Piauí,553,9,4918.0,0.0163,-7.244014,-40.860230


In [43]:
cities = covid_filtered_data[
    [
        "city",
        "population",
        "confirmed",
        "deaths",
        "latitude",
        "longitude"
    ]
]

map = folium.Map(
    location=[-7.092379770410439, -41.45441130138786], 
    zoom_start=6.4, 
    control_scale=False)

for index, location_info in cities.iterrows():
    
    confirmed = str(location_info["confirmed"])
    deaths = str(location_info["deaths"]) 
    population = str(location_info["population"])
    city = str(location_info["city"])

    html_view = folium.IFrame(
        '💀 mortes: ' + deaths + '<br>' + 
        '👥 população: ' + population   
    )

    Popup = folium.Popup(html_view, min_width=200, max_width=500)

    #custon_icon = folium.features.CustomIcon("https://www.iconfinder.com/icons/7847522/download/png/512", icon_size=(18, 18))

    text_icon = folium.DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 10pt; color: black; font-weight: bold"> <img style="height: 12px; width: 12px" src="https://www.iconfinder.com/icons/7847522/download/png/512"> {confirmed}</div>',
        )

    folium.Marker(
        [location_info["latitude"], location_info["longitude"]], 
        popup = Popup,
        tooltip= city,
        icon=text_icon
    ).add_to(map)


map